In [ ]:
!pip install langchain faiss-cpu transformers  PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.9 MB/s eta 0:00:00


In [17]:
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from transformers import pipeline

In [18]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [19]:
pdf_text = extract_text_from_pdf("/content/userstory.pdf")

In [20]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-20-dc99f1f23635>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
chunks = pdf_text.split("\n")

In [22]:
chunk_embeddings = embedding_model.embed_documents(chunks)

In [23]:
faiss_index = faiss.IndexFlatL2(len(chunk_embeddings[0]))  # L2 distance for FAISS
faiss_index.add(np.array(chunk_embeddings))

In [24]:
def retrieve_context(query, faiss_index, embedding_model, chunks, k=3):
    query_embedding = embedding_model.embed_query(query)

    # Search the FAISS index
    distances, indices = faiss_index.search(np.array([query_embedding]), k)

    # Return the top k retrieved chunks
    return [chunks[i] for i in indices[0]]

In [25]:
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [26]:
def answer_question(query):
    # Retrieve context from the document
    context = retrieve_context(query, faiss_index, embedding_model, chunks)

    # Combine context into a single string
    context_text = " ".join(context)

    # Ask the model the question using the context
    result = qa_pipeline(question=query, context=context_text)

    return result["answer"]


In [ ]:
query = "What is discussed in document?"
answer = answer_question(query)
print("Answer:", answer)

Answer: input and  material
